In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain.embeddings.openai import OpenAIEmbeddings


In [3]:
PINECONE_API_KEY = "pcsk_4Wgfrh_GKs4aLeZFcdnveuN8cWLJcEXpKMEKgQ5aQW27iLbcWMwwXR2NAY5jGuuygKgwHQ"
PINECONE_API_ENV = "us-east-1"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("D:\Medical-Chatbot-using-Llama2\data")

In [ ]:
# extracted_data

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


# text_chunks


In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

C:\Users\Shorya\AppData\Local\Temp\ipykernel_3404\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Shorya\anaconda3\envs\mchatbot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [12]:
import os

os.environ["PINECONE_API_KEY"] = "pcsk_4Wgfrh_GKs4aLeZFcdnveuN8cWLJcEXpKMEKgQ5aQW27iLbcWMwwXR2NAY5jGuuygKgwHQ"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"  # or your actual environment


In [16]:
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

docsearch = LangchainPinecone.from_existing_index(
    index_name="medical-bot",
    embedding=embeddings
)


In [18]:
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec

# Step 1: Initialize Pinecone client
pc = Pinecone(api_key="pcsk_4Wgfrh_GKs4aLeZFcdnveuN8cWLJcEXpKMEKgQ5aQW27iLbcWMwwXR2NAY5jGuuygKgwHQ")

# Step 2: Define index name
index_name = "your-index-name"

# Optional: Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # for "all-MiniLM-L6-v2"
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Step 3: Get LangChain-compatible embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Connect LangChain with Pinecone index
docsearch = LangchainPinecone.from_existing_index(index_name=index_name, embedding=embeddings)

# Step 5: Run similarity search
query = "What are allergies?"
docs = docsearch.similarity_search(query, k=3)

# Output results
for i, doc in enumerate(docs, 1):
    print(f"\nResult {i}:\n{doc.page_content}")


In [19]:
query = "What are allergies?"
docs = docsearch.similarity_search(query, k=3)
print(docs)


[]


In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
from langchain.prompts import PromptTemplate

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [22]:
from langchain.llms import CTransformers

llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [23]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa.invoke({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin condition characterized by the formation of pimples, blemishes, and blackheads on the face and other areas of the body. It is caused by a combination of factors, including hormonal changes, excess oil production, clogged pores, and bacterial infections. There are several types of acne, including whiteheads, blackheads, papules, pustules, nodules, and cysts. Acne can be treated with over-the-counter medications, prescription drugs, or professional treatments such as blue light therapy or extractions. It is important to note that acne can have psychological and social impacts on individuals, and it is not just a cosmetic concern.
Response :  Sorry, I don't have enough information to give a detailed answer to your question. Can you provide more context or clarify which part of the problem you are trying to solve?
